# Day 20

link: https://adventofcode.com/2022/day/20

ข้อนี้เราแหกกฎ เปลี่ยนจาก ruby ไปใช้งาน scala เป็นการชั่วคราว สาเหตุเดี๋ยวค่อยว่ากัน

แต่ก่อนอื่น เราอ่าน input ทั้งหมดเก็บเอาไว้ใน List นอกจากนั้นเรายังแปะ index ให้แต่ละตัวเลข เพื่อให้เป็น unique id
ให้เรา track ได้ในกรณีที่มีตัวเลขซ้ำกันใน input

In [1]:
kernel.silent(true)

In [2]:
val input = scala.io.Source.fromFile("in20.txt").getLines.map(_.toLong).zipWithIndex.toList

เราจะใช้วิธี simulate การขยับเปลี่ยนตำแหน่งของตัวเลขอย่างตรงไปตรงมา คือเราจะใส่ input ทั้งหมดลงอีก list อีกอันนึง
แล้วทำตามขั้นตอนต่อไปนี้สำหรับแต่ละ item ใน input list:
- หา index ของ item นี้
- ลบ item นี้ออกจาก list
- คำนวณ index ใหม่ แล้วใส่ item กลับลงไปใน list ที่ index นั้น

และหลังจากทำเสร็จทุก item แล้ว
- หา index ของ item 0
- หา Item ที่อยู่ในตำแหน่งที่ 1000, 2000 และ 3000

คำถามก็คือ เราจะใช้ data structure อะไรจึงจะรองรับ operation เหล่านี้ได้ดี ลองสรุป operation ที่ต้องการอีกที
1. find index by item
2. find item by index
3. remove item
4. insert item at index

เราทำทุก operation ได้ภายใน O(n) แน่ๆ แต่เราจะตั้ง target ให้เร็วกว่านั้น ตั้งไว้ที่ O(log n) ซึ่งก็น่าจะต้องเป็น tree-based structure ซักอย่าง

ลองเริ่มจาก self-balancing binary search tree ... สังเกตว่าขนาดของ left child จะเท่ากับ index ถ้าเราใช้สิ่งนี้เป็น key ของ tree ได้
ก็จะ navigate tree ด้วย index ได้ใน O(log n) ซึ่งก็จะทำให้สามารถทำ operation (2) และ (4) ได้
นอกจากนี้ถ้าเราก็ไม่ต้องมาตาม update index หลังจาก remove item ออกด้วย

แต่เราจำเป็นจะต้องทำ item-based operations ในข้อ 1 และ 3 ให้ได้ด้วย วิธีแก้ของเราก็คือ
สร้าง HashMap ขึ้นมาอีกอันนึง ให้ key เป็น item และ value เป็น index
อันนี้ก็ตอบโจทย์การ lookup ในข้อ (1) และยังเอา index ที่ได้ไปใช้ remove item ในข้อ (3) ได้ด้วย 
แต่ว่าทุกครั้งที่เรา insert หรือ remove จะทำให้ index มันเลื่อน ถึงเราจะไม่ต้องตามแก้ index ใน tree
แต่พอ HashMap มันเก็บ index ก็กลายเป็นว่าเราต้องไปตามแก้ index ใน HashMap เสียเวลาเป็น O(n) อยู่ดี

ดังนั้นมันยังไม่เวิร์ก

เราแก้อีกชั้นนึงด้วยการไม่ใช้ index เป็น key ของ tree ตรงๆ แต่เอาไปผูกกับเลขอีกชุดนึง (เป็น Double) แทน
โดยเราจะการันตีว่า operation ต่างๆ ของ tree จะไม่ทำให้ลำดับของ key เปลี่ยนไป
- เราให้ item แรกสุดใน tree มี key เป็น 0.0 
- ตอน insert ของลง tree ให้ key ของ item ใหม่นี้มีค่าอยู่ตรงกลางระหว่าง item ที่ประกบซ้ายขวา
- ในกรณีที่ insert เข้าไปที่ตำแหน่งหน้าสุดหรือท้ายสุด ก็ให้ key เป็น (fisrt item's key - 1) กับ (last item's key + 1) 
- ตอน remove ของออกจาก tree ไม่ต้องทำอะไรเป็นพิเศษ

ด้วยวิธีนี้ เรายังทำ Operation (2) กับ (4) โดยใช้ index จริงๆ อยู่ และก็ใช้ key ใหม่นี้ทำ operation (1) กับ (3)
ภายใน O(log n) ทั้งหมด

ในส่วนของ implementation เรารู้สึกว่า data structure นี้น่าจะมีโอกาสได้ใช้งานอีก น่าจะทำให้มันดีๆ หน่อยเลยดีกว่า
ก็เลยไปทำใน scala project ที่เอาไว้ทำ Project Euler
ตรงนี้ก็มีปัญหาอีกเล็กน้อยเพราะ self-balancing tree ใน scala ที่มากับ standard lib
มันมีแค่ `mutable.RedBlackTree` ซึ่งถูกซ่อนไว้เป็น private เหมือนเค้าทำมาแบ็ค TreeMap / TreeSet เฉยๆ เลยไม่ expose ออกมาเป็น public
นอกจากนั้น method `size()` ของแต่ละ Node ใน `RedBlackTree` ก็เขียนไว้เป็น O(n) เพราะเค้าไม่ได้ใช้งานจริง เหมือนเขียนไว้ test เฉยๆ
เราก็เลยต้องก็อปโค้ดจาก standard lib มาแก้เองให้มันทำ `size()` ได้ใน O(log n) 

รายละเอียดของ RedBlackTree เราจะข้ามไม่อธิบายละ ขอ import มาใช้เลยละกัน

In [3]:
import $cp.`RBTree.jar`

import io.github.arkorwan.structure.RBTree
import scala.collection.mutable

class BSTIndexedSet[E] {

  private val keyMap = new mutable.HashMap[E, Double]()
  private val rbTree = new RBTree.Tree[Double, E](null)

  private def insertInternal(value: E, weight: Double) = {
    RBTree.insert(rbTree, weight, value)
    keyMap.put(value, weight)
    true
  }

  def append(value: E): Boolean = insert(value, size)

  def insert(value: E, index: Int): Boolean = {
    require(0 <= index && index <= size)
    if (keyMap.contains(value)) {
      false
    } else if (size == 0) {
      insertInternal(value, 0.0)
    } else if (index == size) {
      val leftNode = RBTree.getNodeAtIndex(rbTree.root, index - 1)
      val leftWeight = keyMap(leftNode.value)
      insertInternal(value, leftWeight + 1.0)
    } else if (index == 0) {
      val rightNode = RBTree.getNodeAtIndex(rbTree.root, 0)
      val rightWeight = keyMap(rightNode.value)
      insertInternal(value, rightWeight - 1.0)
    } else {
      val leftNode = RBTree.getNodeAtIndex(rbTree.root, index - 1)
      val leftWeight = keyMap(leftNode.value)
      val rightNode = RBTree.getNodeAtIndex(rbTree.root, index)
      val rightWeight = keyMap(rightNode.value)
      insertInternal(value, (leftWeight + rightWeight) / 2)
    }

  }

  def remove(value: E): Boolean = {
    keyMap.remove(value) match {
      case Some(w) =>
        RBTree.delete(rbTree, w)
        true
      case None =>
        false
    }

  }

  def indexOf(value: E): Int =
    keyMap.get(value) match {
      case None    => -1
      case Some(w) => RBTree.indexOfKey(w, rbTree.root)
    }

  def apply(index: Int): E = RBTree.getNodeAtIndex(rbTree.root, index).value

  def size: Int = keyMap.size

  def toList: List[E] = RBTree.valuesIterator(rbTree).toList

}

## Part 1

มาถึงโจทย์ซักที เอาจริงๆ ก็ไม่เหลือไรมากแล้ว นอกจากจุดที่ต้องระวัง คือถ้าเรามีทั้งหมด n elements การขยับ item ซักอันไปทางขวา (หรือซ้าย) 
จนมันกลับมาอยู่ที่ตำแหน่งเดิม นั้นใช้ n-1 moves ไม่ใช่ n moves ดังนั้นเราต้องเอาจำนวนครั้งที่ต้องขยับมา mod กับ n-1 ไม่ใช่ n

แต่ตอนจบที่โจทย์ถามค่าของตำแหน่งที่ 1000, 2000 และ 3000 ตรงนี้ต้อง mod ด้วย n ตามปกติ

In [4]:
def mix(
        input: Seq[(Long, Int)],
        rounds: Int
    ): BSTIndexedSet[(Long, Int)] = {

      val sortedSet = new BSTIndexedSet[(Long, Int)]()
      input.foreach(sortedSet.append)
      val m = input.length - 1
      (1 to rounds).foreach { _ =>
        input.foreach { e =>
          val currentIndex = sortedSet.indexOf(e)
          val move = ((e._1 + m) % m + m).toInt % m
          val nextIndex = (currentIndex + move) % m
          sortedSet.remove(e)
          sortedSet.insert(e, nextIndex)
        }
      }
      sortedSet
    }

val sortedSet = mix(input, 1)
val zero = sortedSet.indexOf(input.find(_._1 == 0).get)
val res = Seq(1000, 2000, 3000).map{i => sortedSet((zero + i) % input.length)._1}.sum
println(res)

7228


## Part 2

เหลือแค่ทำเหมือนเดิมวน 10 รอบ ง่ายเลย ของยากเราทำไปหมดแล้วแต่แรก

In [5]:
val keyedInput = input.map{ case (x, y) => (x * 811589153L, y)}

val sortedSet = mix(keyedInput, 10)
val zero = sortedSet.indexOf(keyedInput.find(_._1 == 0).get)
val res = Seq(1000, 2000, 3000).map{i => sortedSet((zero + i) % keyedInput.length)._1}.sum
println(res)

4526232706281
